# Recuperação da Informação e Busca na Web - 2018.1

### Atividade: Lab 02 - Parte 1 - Expansão de Consultas
### Aluno: Johanny de Lucena Santos

## - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

## Importações necessárias para a análise dos dados

In [1]:
import pandas as pd
import nltk
import numpy as np
import ast
import re
from unicodedata import normalize
from collections import Counter
import math

from scipy import sparse
from nltk import bigrams
import scipy.sparse as sps

### Fazendo download de biblioteca necessária para realizar a tokenização das palavras

In [72]:
#nltk.download('punkt')



## Definindo o arquivo com os dados a ser analisado e o gabarito

In [2]:
dataset = pd.read_csv('../data/estadao_noticias_eleicao.csv', sep=',', encoding="utf-8")

#Substituindo linhas com valores NaN por string vazia, para não atrapalhar nas operações
dataset = dataset.replace(np.nan, '', regex=True)

## Tratamento do Gabarito
 Usando a biblioteca **Abstract Syntax Trees** (ast) para converter as strings do gabarito para o objeto lista, junto com a função apply() da biblioteca **Pandas** para aplicar a conversão em todas as linhas

In [4]:
gabarito = pd.read_csv('../data/gabarito.csv')

gabarito['google']        = gabarito['google'].apply(ast.literal_eval)
gabarito['busca_binaria'] = gabarito['busca_binaria'].apply(ast.literal_eval)
gabarito['tf']            = gabarito['tf'].apply(ast.literal_eval)
gabarito['tfidf']         = gabarito['tfidf'].apply(ast.literal_eval)
gabarito['bm25']          = gabarito['bm25'].apply(ast.literal_eval)

## Funções para avaliação das saídas utilizando a métrica Mean Average Precision (MAP)

In [5]:
def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

### Função para remover acentuação das palavras dos documentos
Módulo **re** e **unicodedata** necessários para tratamento de expressões regulares e normalização de string

In [6]:
def remove_acentuacao(palavra):
    pattern = re.compile('[^a-zA-Z0-9 ]')
    palavra = normalize('NFKD', palavra).encode('ASCII', 'ignore').decode('ASCII')
    return pattern.sub(' ', palavra)

## Realizando junção dos títulos, subtítulos e conteúdos
Na junção, está sendo removidos todos os acentos para facilitar a criação dos tokens dos dados

In [7]:
documentos = dataset.titulo + " " + dataset.subTitulo + " " + dataset.conteudo
documentos = documentos.apply(lambda palavra: remove_acentuacao(palavra).lower())

## Matrix and Vocabulary construction

In [8]:
def co_occurrence_matrix(corpus):
    vocab = set(corpus)
    vocab = list(vocab)
    n = len(vocab)
   
    vocab_to_index = {word:i for i, word in enumerate(vocab)}
    
    bi_grams = list(bigrams(corpus))

    bigram_freq = nltk.FreqDist(bi_grams).most_common(len(bi_grams))

    I=list()
    J=list()
    V=list()
    
    for bigram in bigram_freq:
        current = bigram[0][1]
        previous = bigram[0][0]
        count = bigram[1]

        I.append(vocab_to_index[previous])
        J.append(vocab_to_index[current])
        V.append(count)
        
    co_occurrence_matrix = sparse.coo_matrix((V,(I,J)), shape=(n,n))

    return co_occurrence_matrix, vocab_to_index

## Criação dos docIDs

In [9]:
docIDs = dataset.idNoticia

## Criando os tokens dos dados e a frequência de termos de cada documento da lista de postings
Utilizando a função **Count** da biblioteca **Collections** para realizar a contagem dos termos (frequência)

In [26]:
tokens = documentos.apply(nltk.word_tokenize)
tokens_list = documentos.apply(lambda text: text.lower().split())
tokens2 = [token for tokens_list in tokens_list for token in tokens_list]
termo_frequency = tokens.apply(Counter)

In [28]:
matrix, vocab = co_occurrence_matrix(tokens2)

## Consult bigram frequency

In [30]:
consultable_matrix = matrix.tocsr()

In [31]:
def consult_frequency(w1, w2):
    return (consultable_matrix[vocab[w1], vocab[w2]])

In [45]:
def top_3_words(termo):
    ranking = []
    for termo_matrix in consultable_matrix:
        ranking.append(consult_frequency(termo, termo_matrix))
    
    return sorted(ranking, reverse=True)[0:3]

In [55]:
print(vocab['poucos'])
print(vocab['recursos'])
print(type(consultable_matrix))

25981
11196
<class 'scipy.sparse.csr.csr_matrix'>


## Função geradora do índice invertido

In [12]:
indiceInvertido = {}

def gerarIndiceInvertido():
    for i in range(len(tokens)):
        idNoticia = docIDs[i]
        palavras = tokens[i]
    
        for palavra in palavras:
            if palavra not in indiceInvertido:
                indiceInvertido[palavra.lower()] = {}
        
            if not indiceInvertido[palavra.lower()].get(idNoticia):
                docs = indiceInvertido[palavra.lower()]
                docs[idNoticia] = termo_frequency[i][palavra.lower()]

In [13]:
gerarIndiceInvertido()

## Função para retornar as palavras mais proximas

In [14]:
def busca_aproximada(termo, qtd):
    termos = indiceInvertido.keys()
    return sorted(termos, key=lambda palavra: nltk.edit_distance(termo.lower(),palavra))[0:qtd]

## Função para gerar um dicionário com os pesos dos índices

In [15]:
def gerar_dict_pesos(frase, gerador_peso):
    termos = frase.split(" ")
    docs_peso = {}
    
    for i in range(len(termos)):
        termo = termos[i].lower()
        docs = indiceInvertido[termo]
        
        for doc_id in docs:
            tf = docs[doc_id]
            
            if doc_id not in docs_peso:
                docs_peso[doc_id] = np.array([0 if j != i else gerador_peso(tf) for j in range(len(termos))])
            else:
                doc_vector = docs_peso[doc_id]
                docs_peso[doc_id] = np.array([doc_vector[j] if j != i else gerador_peso(tf) for j in range(len(termos))])
    
    return docs_peso

## Função para gerar um dicionário com vetores binários

In [16]:
def gerar_binario(frase):
    def gerador_peso(tf):
        return 1
    return gerar_dict_pesos(frase, gerador_peso)

## Função para gerar o TF da consulta

In [17]:
def gerar_vetor_tf(frase):
    def gerador_peso(tf):
        return tf
    return gerar_dict_pesos(frase, gerador_peso)

## Função para gerar IDF dos termos

In [18]:
def gerar_vetor_tfidf(frase):
    termos = frase.split(" ")
    vetor_tfidf = np.array([math.log((len(documentos)+1)/len(indiceInvertido[termo.lower()])) for termo in termos])
    return vetor_tfidf

## Função para gerar o vetor binário de consulta

In [19]:
def gerar_consulta(frase):
    termos = frase.split(" ")
    vetor = np.array([1 if indiceInvertido.get(termo.lower()) else 0 for termo in termos])
    return vetor

## Função para gerar um dicionário com vetores BM25

In [20]:
def gerar_vetor_bm25(frase):
    docs_tf = gerar_vetor_tf(frase)
    k = 5
    vetor_bm25 = {doc_id: np.array([((k+1)*tf)/(tf+k) for tf in vetor_tf]) for doc_id, vetor_tf in docs_tf.items()}
    
    return vetor_bm25

## Função de busca binária

In [21]:
def busca_binaria(frase):
    docs_tf = gerar_binario(frase)
    consulta = gerar_consulta(frase)
    doc_rank = sorted(list(docs_tf.items()), key=lambda doc: np.dot(doc[1], consulta), reverse=True)[:5] 
    return [doc[0] for doc in doc_rank]

## Função de busca por TF

In [22]:
def busca_tf(frase):
    docs_tf = gerar_vetor_tf(frase)
    query = gerar_consulta(frase)
    doc_rank = sorted(list(docs_tf.items()), key=lambda doc: np.dot(doc[1], query), reverse=True)[:5] 
    return [doc[0] for doc in doc_rank]

## Função de busca por TF IDF

In [23]:
def busca_tf_idf(frase):
    doc_tf = gerar_vetor_tf(frase)
    doc_idf = gerar_vetor_tfidf(frase)
    doc_rank = sorted(list(doc_tf.items()), key=lambda doc: np.dot(doc[1], doc_idf), reverse=True)[:5]
    return [doc[0] for doc in doc_rank]

## Função de busca por BM25

In [24]:
def busca_bm25(frase):
    doc_bm25 = gerar_vetor_bm25(frase)
    consulta = gerar_consulta(frase)
    doc_rank = sorted(list(doc_bm25.items()), key=lambda doc: np.dot(doc[1], consulta), reverse=True)[:5]
    return [doc[0] for doc in doc_rank]

# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

## Exemplo consult frequency

In [32]:
w1 = 'poucos'
w2 = 'recursos'
consult_frequency(w1,w2)

3

In [46]:
top_3_words('poucos')

TypeError: unhashable type: 'csr_matrix'

In [49]:
tst = []
for term in consultable_matrix:
    tst.append(term)

print(tst[0])

  (0, 693)	16
  (0, 2631)	5
  (0, 2752)	2
  (0, 3118)	4
  (0, 4719)	1
  (0, 5836)	4
  (0, 5850)	9
  (0, 7106)	1
  (0, 7984)	1
  (0, 12977)	1
  (0, 13866)	1
  (0, 15175)	1
  (0, 15418)	2
  (0, 19078)	1
  (0, 19655)	1
  (0, 22422)	1
  (0, 25894)	1
  (0, 27039)	1
  (0, 29645)	3
  (0, 31293)	1
  (0, 31321)	1
  (0, 32247)	1
  (0, 33379)	6
  (0, 35181)	1
  (0, 35402)	2
  (0, 40463)	1
  (0, 42548)	3
  (0, 45814)	1
  (0, 47352)	1
  (0, 50246)	1
  (0, 50771)	1
